In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from skportfolio import *
from skportfolio.datasets import *
from skportfolio.backtest.backtester import Strategy, Backtester
from skportfolio.datasets import load_dataset, get_dataset_names

In [ ]:
%config InlineBackend.figure_format = 'svg' # makes the plots HD in the notebook
mpl.rcParams["figure.autolayout"] = True # enables tigh layout. Better multiplots
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'

In [ ]:
prices = load_dataset('dowportfolio')
prices.index = pd.to_datetime(prices.index)
prices = prices.resample('B').agg('first')
prices = prices[prices.isna().sum(1)==0]

In [ ]:
strategy = Strategy(
    initial_weights=EquallyWeighted().fit(prices).weights_,
    initial_portfolio_value=10000,
    estimator=EquallyWeighted(),
    rebalance_frequency='BM',
    lookback_periods=(40,126),
    transaction_costs=(0.05, 0.05) # buy and sell costs
)

In [ ]:
backtester = Backtester(strategy=strategy, warmup_period=40)
backtester.fit(prices);

In [ ]:
backtester.turnover_.dropna()

In [ ]:
backtester.positions_

In [ ]:
backtester.equity_.dropna().loc['2006-02-25':].plot()

In [ ]:
equity_curve(backtester.equity_.dropna().loc['2006-03-01':],initial_value=1).plot()